# Ejercicios de Delay Flights para trabajar con dates

In [1]:
# Crear sesión
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession
        .builder
        .appName("DelayFlightsPySpark")
        .getOrCreate())

Para empezar importamos los datos desde el csv de nuestra carpeta de descargas.  
Ya que al inferir el Schema automaticamente puede dar problemas, lo añadimos manualmente.  
Atención a la columna _date_, que la guardamos como un tipo string en vez de __Date__ o __timestamp__.

In [2]:
# Importar datos
from pyspark.sql.types import StructType, IntegerType, StringType
ruta = "data/departuredelays.csv"
schema = (StructType()
         .add("date", StringType(), True)
         .add("delay", IntegerType(), True)
         .add("distance", IntegerType(), True)
         .add("origin", StringType(), True)
         .add("destination", StringType(), True))
          
df_raw = (spark.read.csv(ruta,
                        header = True,
                        schema = schema))
df_raw.show()
df_raw.printSchema()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061215|   -6|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01061230|    0|     369|   ABE|        DTW|
|01060625|   -3|     602|   ABE|        ATL|
|01070600|    0|     369|   ABE|        DTW|
|01071725|    0|     602|   ABE|        ATL|
|01071230|    0|     369|   ABE|        DTW|
|01070625|    0|     602|   ABE|        ATL|
|01071219|    0|     569|   ABE|        ORD|
|01080600|

Debido a que queremos tratar la columna _date_ como si fueran fechas en vez de un string, la convertimos a formato. __timestamp__  
* __Date__ solo incluye las partes de año, mes y día.  
* __timestamp__ incluye año, mes, día, hora, minuto, segundo y milisegundo.  

In [3]:
# Cambiar a formato timestamp
from pyspark.sql.functions import to_timestamp
df = (df_raw
      .withColumn("date", to_timestamp("date", 'MMddHHmm').cast('timestamp')))
df.show()
df.printSchema()

+-------------------+-----+--------+------+-----------+
|               date|delay|distance|origin|destination|
+-------------------+-----+--------+------+-----------+
|1970-01-01 12:45:00|    6|     602|   ABE|        ATL|
|1970-01-02 06:00:00|   -8|     369|   ABE|        DTW|
|1970-01-02 12:45:00|   -2|     602|   ABE|        ATL|
|1970-01-02 06:05:00|   -4|     602|   ABE|        ATL|
|1970-01-03 12:45:00|   -4|     602|   ABE|        ATL|
|1970-01-03 06:05:00|    0|     602|   ABE|        ATL|
|1970-01-04 12:43:00|   10|     602|   ABE|        ATL|
|1970-01-04 06:05:00|   28|     602|   ABE|        ATL|
|1970-01-05 12:45:00|   88|     602|   ABE|        ATL|
|1970-01-05 06:05:00|    9|     602|   ABE|        ATL|
|1970-01-06 12:15:00|   -6|     602|   ABE|        ATL|
|1970-01-06 17:25:00|   69|     602|   ABE|        ATL|
|1970-01-06 12:30:00|    0|     369|   ABE|        DTW|
|1970-01-06 06:25:00|   -3|     602|   ABE|        ATL|
|1970-01-07 06:00:00|    0|     369|   ABE|     

Podemos utilizar diferentes funciones para obtener solo una parte de la fecha, esto resulta muy util a la hora de filtrar:  
* __Year__ para obtener el año.
* __Month__ para obtener el mes.
* __Dayofmonth__ para obtener el día del mes.
* __Dayofweek__ para obtener el día de la semana.
* __Dayofyear__ para obtener el día del año.
* __Hour__ para obtener la hora.
* __Minute__ para obtener el minuto.
* __Second__ para obtener el segundo y milisegundos si multiplicas por mil.

In [4]:
# Filtrar por mes
from pyspark.sql.functions import month
df.filter(month("date") == 2).show()

+-------------------+-----+--------+------+-----------+
|               date|delay|distance|origin|destination|
+-------------------+-----+--------+------+-----------+
|1970-02-01 12:15:00|   -8|     602|   ABE|        ATL|
|1970-02-01 06:00:00|   -4|     369|   ABE|        DTW|
|1970-02-01 06:25:00|   -1|     602|   ABE|        ATL|
|1970-02-02 12:15:00|   -5|     602|   ABE|        ATL|
|1970-02-02 17:25:00|   -9|     602|   ABE|        ATL|
|1970-02-03 12:15:00|  158|     602|   ABE|        ATL|
|1970-02-03 06:00:00|    8|     369|   ABE|        DTW|
|1970-02-03 17:25:00|   76|     602|   ABE|        ATL|
|1970-02-03 12:30:00|   30|     369|   ABE|        DTW|
|1970-02-03 06:25:00|   22|     602|   ABE|        ATL|
|1970-02-03 12:19:00|   30|     569|   ABE|        ORD|
|1970-02-04 06:00:00|    6|     369|   ABE|        DTW|
|1970-02-04 17:25:00|   -4|     602|   ABE|        ATL|
|1970-02-04 12:30:00|   -2|     369|   ABE|        DTW|
|1970-02-04 06:25:00|    5|     602|   ABE|     

In [5]:
# Filtrar por mes
from pyspark.sql.functions import month
df.filter(month("date") > 2).show()

+-------------------+-----+--------+------+-----------+
|               date|delay|distance|origin|destination|
+-------------------+-----+--------+------+-----------+
|1970-03-01 06:05:00|   -4|     369|   ABE|        DTW|
|1970-03-01 06:25:00|   -6|     602|   ABE|        ATL|
|1970-03-02 17:15:00|    1|     369|   ABE|        DTW|
|1970-03-02 12:45:00|   -6|     369|   ABE|        DTW|
|1970-03-02 17:25:00|  275|     602|   ABE|        ATL|
|1970-03-02 06:00:00|   -2|     602|   ABE|        ATL|
|1970-03-03 06:10:00|    0|     369|   ABE|        DTW|
|1970-03-03 17:15:00|   86|     369|   ABE|        DTW|
|1970-03-03 12:45:00|   -1|     369|   ABE|        DTW|
|1970-03-03 12:06:00|   -2|     602|   ABE|        ATL|
|1970-03-03 17:25:00|    0|     602|   ABE|        ATL|
|1970-03-03 05:45:00|   -8|     602|   ABE|        ATL|
|1970-03-03 16:28:00|    0|     569|   ABE|        ORD|
|1970-03-04 06:10:00|    0|     369|   ABE|        DTW|
|1970-03-04 17:15:00|   15|     369|   ABE|     

In [6]:
# Filtrar por hora
from pyspark.sql.functions import hour
df.filter(hour("date") > 12).show()

+-------------------+-----+--------+------+-----------+
|               date|delay|distance|origin|destination|
+-------------------+-----+--------+------+-----------+
|1970-01-06 17:25:00|   69|     602|   ABE|        ATL|
|1970-01-07 17:25:00|    0|     602|   ABE|        ATL|
|1970-01-09 17:25:00|    0|     602|   ABE|        ATL|
|1970-01-10 17:25:00|    7|     602|   ABE|        ATL|
|1970-01-12 17:25:00|   -1|     602|   ABE|        ATL|
|1970-01-13 17:25:00|   -6|     602|   ABE|        ATL|
|1970-01-14 17:25:00|   -9|     602|   ABE|        ATL|
|1970-01-15 17:25:00|   -6|     602|   ABE|        ATL|
|1970-01-16 17:25:00|   -6|     602|   ABE|        ATL|
|1970-01-17 17:25:00|    5|     602|   ABE|        ATL|
|1970-01-19 17:25:00|   -5|     602|   ABE|        ATL|
|1970-01-20 17:25:00|   -5|     602|   ABE|        ATL|
|1970-01-21 17:25:00|    0|     602|   ABE|        ATL|
|1970-01-23 17:25:00|  180|     602|   ABE|        ATL|
|1970-01-24 17:25:00|    2|     602|   ABE|     

También se pueden sumar valores a la fecha, incluso de manera dinámica.  
* __unix_timestamp__ convierte del formato timestamp a los segundos pasados desde el "epoch" (01/01/1970) lo cual nos permite trabajar con las fechas.  
* atención con el _delay_, que está en minutos y no en segundos.

In [7]:
# Añadir dinamicamente el delay a date
from pyspark.sql.functions import unix_timestamp
dfDelays = (df.withColumn("delayed_date", (unix_timestamp(df.date) + df.delay * 60).cast('timestamp')))
dfDelays.show()

+-------------------+-----+--------+------+-----------+-------------------+
|               date|delay|distance|origin|destination|       delayed_date|
+-------------------+-----+--------+------+-----------+-------------------+
|1970-01-01 12:45:00|    6|     602|   ABE|        ATL|1970-01-01 12:51:00|
|1970-01-02 06:00:00|   -8|     369|   ABE|        DTW|1970-01-02 05:52:00|
|1970-01-02 12:45:00|   -2|     602|   ABE|        ATL|1970-01-02 12:43:00|
|1970-01-02 06:05:00|   -4|     602|   ABE|        ATL|1970-01-02 06:01:00|
|1970-01-03 12:45:00|   -4|     602|   ABE|        ATL|1970-01-03 12:41:00|
|1970-01-03 06:05:00|    0|     602|   ABE|        ATL|1970-01-03 06:05:00|
|1970-01-04 12:43:00|   10|     602|   ABE|        ATL|1970-01-04 12:53:00|
|1970-01-04 06:05:00|   28|     602|   ABE|        ATL|1970-01-04 06:33:00|
|1970-01-05 12:45:00|   88|     602|   ABE|        ATL|1970-01-05 14:13:00|
|1970-01-05 06:05:00|    9|     602|   ABE|        ATL|1970-01-05 06:14:00|
|1970-01-06 

Existe también la opción de solo mostrar ciertas partes del __date__ o cambiar su estructura.  
Esto viene bien si hay ciertas partes que no aportan información o queremos deshacernos de ellas (como aquí con el año y los segundos).

In [8]:
# Mostrar solo partes del date
from pyspark.sql.functions import date_format
df_clean = (df.select(date_format(df.date, 'MM-dd HH:mm').alias("date"), df.delay, df.distance, df.origin, df.destination))
df_clean.show()

+-----------+-----+--------+------+-----------+
|       date|delay|distance|origin|destination|
+-----------+-----+--------+------+-----------+
|01-01 12:45|    6|     602|   ABE|        ATL|
|01-02 06:00|   -8|     369|   ABE|        DTW|
|01-02 12:45|   -2|     602|   ABE|        ATL|
|01-02 06:05|   -4|     602|   ABE|        ATL|
|01-03 12:45|   -4|     602|   ABE|        ATL|
|01-03 06:05|    0|     602|   ABE|        ATL|
|01-04 12:43|   10|     602|   ABE|        ATL|
|01-04 06:05|   28|     602|   ABE|        ATL|
|01-05 12:45|   88|     602|   ABE|        ATL|
|01-05 06:05|    9|     602|   ABE|        ATL|
|01-06 12:15|   -6|     602|   ABE|        ATL|
|01-06 17:25|   69|     602|   ABE|        ATL|
|01-06 12:30|    0|     369|   ABE|        DTW|
|01-06 06:25|   -3|     602|   ABE|        ATL|
|01-07 06:00|    0|     369|   ABE|        DTW|
|01-07 17:25|    0|     602|   ABE|        ATL|
|01-07 12:30|    0|     369|   ABE|        DTW|
|01-07 06:25|    0|     602|   ABE|     

In [9]:
# Añadir delayed_date y mostrar solo la información importante
from pyspark.sql.functions import date_format
dfDelays_clean = (dfDelays.select(date_format(dfDelays.date, 'MM-dd HH:mm').alias("date"),
                                  dfDelays.delay,
                                  dfDelays.distance,
                                  df.origin,
                                  dfDelays.destination,
                                  date_format(dfDelays.delayed_date, 'MM-dd HH:mm').alias("delayed_date")))
dfDelays_clean.show()

+-----------+-----+--------+------+-----------+------------+
|       date|delay|distance|origin|destination|delayed_date|
+-----------+-----+--------+------+-----------+------------+
|01-01 12:45|    6|     602|   ABE|        ATL| 01-01 12:51|
|01-02 06:00|   -8|     369|   ABE|        DTW| 01-02 05:52|
|01-02 12:45|   -2|     602|   ABE|        ATL| 01-02 12:43|
|01-02 06:05|   -4|     602|   ABE|        ATL| 01-02 06:01|
|01-03 12:45|   -4|     602|   ABE|        ATL| 01-03 12:41|
|01-03 06:05|    0|     602|   ABE|        ATL| 01-03 06:05|
|01-04 12:43|   10|     602|   ABE|        ATL| 01-04 12:53|
|01-04 06:05|   28|     602|   ABE|        ATL| 01-04 06:33|
|01-05 12:45|   88|     602|   ABE|        ATL| 01-05 14:13|
|01-05 06:05|    9|     602|   ABE|        ATL| 01-05 06:14|
|01-06 12:15|   -6|     602|   ABE|        ATL| 01-06 12:09|
|01-06 17:25|   69|     602|   ABE|        ATL| 01-06 18:34|
|01-06 12:30|    0|     369|   ABE|        DTW| 01-06 12:30|
|01-06 06:25|   -3|     

Del mismo modo que podíamos sumar numeros al __date__, tenemos la opción de restar dates para obtener valores.  
  
Esto es tan sencillo como convertir el formato __date__/__timestamp__ a un valor numérico y operar con él.  
Antes para ello hemos utilizado __unix_timestamp__ pero también podemos hacer un cast a __long__, por ejemplo.

In [16]:
# Volver a sacar el delay con los 2 dates
from pyspark.sql.functions import datediff
(dfDelays.select(dfDelays.date,
                 dfDelays.delayed_date,
                 dfDelays.delay,
                 ((dfDelays.delayed_date.cast("long") - dfDelays.date.cast("long")) / 60).cast("int").alias("delay2"))
 .show())

+-------------------+-------------------+-----+------+
|               date|       delayed_date|delay|delay2|
+-------------------+-------------------+-----+------+
|1970-01-01 12:45:00|1970-01-01 12:51:00|    6|     6|
|1970-01-02 06:00:00|1970-01-02 05:52:00|   -8|    -8|
|1970-01-02 12:45:00|1970-01-02 12:43:00|   -2|    -2|
|1970-01-02 06:05:00|1970-01-02 06:01:00|   -4|    -4|
|1970-01-03 12:45:00|1970-01-03 12:41:00|   -4|    -4|
|1970-01-03 06:05:00|1970-01-03 06:05:00|    0|     0|
|1970-01-04 12:43:00|1970-01-04 12:53:00|   10|    10|
|1970-01-04 06:05:00|1970-01-04 06:33:00|   28|    28|
|1970-01-05 12:45:00|1970-01-05 14:13:00|   88|    88|
|1970-01-05 06:05:00|1970-01-05 06:14:00|    9|     9|
|1970-01-06 12:15:00|1970-01-06 12:09:00|   -6|    -6|
|1970-01-06 17:25:00|1970-01-06 18:34:00|   69|    69|
|1970-01-06 12:30:00|1970-01-06 12:30:00|    0|     0|
|1970-01-06 06:25:00|1970-01-06 06:22:00|   -3|    -3|
|1970-01-07 06:00:00|1970-01-07 06:00:00|    0|     0|
|1970-01-0

In [11]:
# Calcular el delay total por origen y destino
df_grouped_od = (df.select("origin", "destination", "delay")
             .groupBy("origin", "destination")
             .sum("delay")
             .withColumnRenamed("sum(delay)", "total_delay")
             .orderBy("origin", "destination"))

df_grouped_od.show()

+------+-----------+-----------+
|origin|destination|total_delay|
+------+-----------+-----------+
|   ABE|        ATL|       2748|
|   ABE|        DTW|       1158|
|   ABE|        ORD|       1207|
|   ABI|        DFW|       5128|
|   ABQ|        ATL|       1852|
|   ABQ|        BWI|        695|
|   ABQ|        DAL|       8610|
|   ABQ|        DEN|       4135|
|   ABQ|        DFW|       3966|
|   ABQ|        HOU|       3539|
|   ABQ|        IAD|        280|
|   ABQ|        IAH|       2926|
|   ABQ|        JFK|        789|
|   ABQ|        LAS|       6151|
|   ABQ|        LAX|       8085|
|   ABQ|        MCI|        888|
|   ABQ|        MDW|       2342|
|   ABQ|        MSP|         47|
|   ABQ|        OAK|       3971|
|   ABQ|        ORD|       1580|
+------+-----------+-----------+
only showing top 20 rows



In [12]:
# Calcular el delay total por dia y mes
from pyspark.sql.functions import month, dayofmonth
df_grouped_dm = (df.select(month("date").alias("month"),
                           dayofmonth("date").alias("day"),
                           "delay")
                 .filter(df.delay < 0)
             .groupBy("month", "day")
             .sum("delay")
             .withColumnRenamed("sum(delay)", "total_delay")
             .orderBy("month", "day"))

df_grouped_dm.show()
df_grouped_dm.printSchema()

+-----+---+-----------+
|month|day|total_delay|
+-----+---+-----------+
|    1|  1|     -26079|
|    1|  2|     -19373|
|    1|  3|     -17628|
|    1|  4|     -13464|
|    1|  5|     -12640|
|    1|  6|     -13071|
|    1|  7|     -15629|
|    1|  8|     -20911|
|    1|  9|     -28199|
|    1| 10|     -29608|
|    1| 11|     -25413|
|    1| 12|     -39531|
|    1| 13|     -49322|
|    1| 14|     -49566|
|    1| 15|     -44655|
|    1| 16|     -39108|
|    1| 17|     -38876|
|    1| 18|     -34209|
|    1| 19|     -46473|
|    1| 20|     -45128|
+-----+---+-----------+
only showing top 20 rows

root
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- total_delay: long (nullable = true)

